In [50]:
#### evaluation for error sources #####
import numpy as np
import abc
# data generator for gridded scans
from ultrasonic_imaging_python.forward_models.data_synthesizers import DataGeneratorForwardModel3D
# data generator for manual scans
from ultrasonic_imaging_python.forward_models.data_synthesizers_manual_scan import DataGeneratorManualScanWithoutError
# reconstruction 
from ultrasonic_imaging_python.reconstruction.saft_for_manual_scan import SAFTonGridforRandomScan
# tools
from ultrasonic_imaging_python.definitions import units
ureg = units.ureg

"""
Workflow
1. load json param_dict files in the main.py (params_const, params_variable, err_sources)
2. call the appropriate ErrorSourceEvaluator
3. choose the appropriate specimen variables (except sub-vars)
4. for-loop : sub-vars
5. for-loop : main-var
6. set file names
7. call the DataGeneratorForwardModel3D / DataGeneratorManualScan class
8. set parameters
9. configure logger
10. generate data
11. call the SAFTEngineProgressiveUpdate3D / SAFTonGridforRandomScan class 
12. configure logger
13. get reco
14. get cimage
15. save the data (A-scans, reco, cimage in txt)
16. log the param_dict (txt?)

"""


class ErrorSourceEvaluator(abc.ABC):
    
    # constructor
    def __init__(self):
        """ Constructor
        
        Parameters
        ----------
            None

        """
        super().__init__() 
        
        # parameter setting
        self.measurement_param_dict = {}
        self.pulse_param_dict = {}
        
    def _set_file_names(self):
        pass

########################################################################################## logger configuration #######     
    def _configure_logger(self, fname_log, fname_json_const, fname_json_var, var_set):
        """ configuration for logging the program/data. The data is logged, only when the data is saved. 
        (i.e. save_data == True)

        Parameters
        ----------
            fname_log : str, name of the log file
            fname_json_const : str, name of the json file containing constant parameters
            fname_json_var: str, name of the json file, containing variables
            var_set : dict, suggesting which variable sets were selected for generating the data
        """
        
        # setup info to pack into the logger
        self.fname_json_const = str(fname_json_const)
        self.fname_json_var = str(fname_json_var)
        self.var_set = dict(var_set)
        
        # logger setup
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)
        
        formatter = logging.Formatter('%(asctime)s:%(levelname)s:%(name)s:%(message)s')
        
        self.file_handler = logging.FileHandler(fname_log)
        self.file_handler.setFormatter(formatter)
        
        self.logger.addHandler(self.file_handler)

        
############################################################################### load param_dict from json files #######
    @abc.abstractmethod
    def set_parameter_dataset(self):
        """ 
        What this function should do:
            1. choose appropriate specimen variables (except sub-vars)
            2. for loop with sub-vars
            3. for loop with main-vars
            4. call _set_file_names
            5. call all _register... fuctions 
            6. call the proper DataGenerator class
            7. call set_measurement_parameters() from the DataGenerator classes
        """

    ### setup for specimen constant ###
    def _register_specimen_constant(self, specimen_constant):
        """ with this funciton the parameter dictionary can be obtained from the selected json file
        
        Parameters 
        ----------
            specimen_constant : dict containing following two dictionaries...
                specimen_constant : dict
                    containing values of c0, fS, openingangle, anglex, angley, t0, xImage, yImage, zImage
                pulse_params_default : dict
                    containing info on pulse_model_name, number of pulse (N_pulse) and fCarrier
            
        Reutuns
        -------
            None
                
        newly set parameters (not returned) 
        -----------------------------------
            self.measurement_param_dict : dict (with unit)
                containing c0, fS, openingangle, anglex, angley, t0, xImage, yImage, zImage
            self.raw_pulse_params : dict (with unit)
                containing tPulse, fCarrier, fS and B
            pulse_model_name : str
                
        """
        # ureg is not converted properly from json files, thus unit should be added here for FWM
        self.measurement_param_dict = self._add_unit_to_parameters(specimen_constant['specimen_constant'],
                                                                   self.measurement_param_dict)
        self.pulse_param_dict = self._add_unit_to_parameters(specimen_constant['pulse_params_default'],
                                                             self.measurement_param_dict)
        
        # update pulse_params
        self.pulse_param_dict.update({
                    'tPulse': self.pulse_param_dict['N_pulse'] / self.measurement_param_dict['fS'],
                    'fS': self.measurement_param_dict['fS'],
                    })
        # pulse_param_dict may contain only required parameters due to the pulse_models class, 
        # that's why N_pulse should be eliminated
        del self.pulse_param_dict['N_pulse']
        
        #register pulse model name
        self.pulse_model_name = specimen_constant['pulse_params_default']['pulse_model_name']
        
        
    ### setup for specimen variables ###    
    def _register_specimen_variables(self, specimen_variables):
        """ with this funciton the parameter dictionary can be obtained from the selected json file
        
        Parameters 
        ----------
            specimen_variables : dict
                containing dimension (Nxdata, Nydata & Ntdata), defect_map and scan_path_files (t0 = optional),
                which should be specified before input.
                e.g. : specimen_variables[dimension] should contain only one Nxdata, Nydata and Ntdata
            err_sources : dict
                -> in progress
            
        Reutuns
        -------
            None
                
        newly set parameters (not returned) 
        -----------------------------------
            self.measurement_param_dict : dict (with unit)
                containing c0, fS, openingangle, anglex, angley, t0, xImage, yImage, zImage + 
                Nxdata, Nydata, Ntdata, defect_map(unitless) and scan_path_files
            
        """
               
        self.measurement_param_dict.update({
                'Nxdata': int(specimen_variables['dimension']['Nxdata']), 
                'Nydata': int(specimen_variables['dimension']['Nydata']),
                'Ntdata': int(specimen_variables['dimension']['Ntdata']),
                'defect_map' : list(specimen_variables['defect_map']),
                'scan_path_file' : str(specimen_variables['scan_path_file'])
                })
        

    #### add defect positions (with unit) to the self.measurement_param_dict ####
    def _register_defect_positions(self):
        """ with this function the defect map (unitless) is converted to the pos_defect(with unit).
        
        Returns
        -------
            None, but the self.measurement_param_dict is updated with 'pos_defect'. 

        """
        
        dz = (self.measurement_param_dict['c0'].to_base_units().magnitude / 
              (2.0 * self.measurement_param_dict['fS'].to_base_units().magnitude))* 10**3
        pos_defect = np.array(self.measurement_param_dict['defect_map'], float)
        pos_defect[:, 2] = pos_defect[:, 2]* dz
        # add defect positions to the parameter dictionary        
        self.measurement_param_dict.update({
                'pos_defect' : pos_defect* ureg.millimeter
                })
        
    ### setup for error sources ###
    @abc.abstractmerhod
    def _register_error_source_variables(self, err_sources):
        """
        what this function should do:
            1. 
        """
        

    ###### check unit of parameters in json data #########
    def _add_unit_to_parameters(self, from_which_dict, to_which_dict):
        """ with this function the format (i.e. unit) of the parameters in json files is examined and the 
        given parameter dictionary is updated accordingly (with unit).

        Parameters
        ----------
            from_which_dict : dict,
                the parameter dictionary (created from json files) from which the parameters are taken
            to_which_dict : dict
                the parameter dictionray to which these parameters with unit are added
        
        Returns
        -------
            to_which_dict(with unit)
        
        """
        copy_dict = from_which_dict
        unit_dict = {
                'c0' : ureg.meter / ureg.second,
                'fS' : ureg.megahertz,
                'zImage': ureg.meter,
                'xImage': ureg.meter,
                'yImage': ureg.meter,
                'fCarrier' : ureg.megahertz,
                'dxdata' : ureg.millimeter,
                'dydata' : ureg.millimeter,
                't0' : ureg.second,
                'fCarrier' : ureg.megahertz,
                }
        
        for key in copy_dict:
            param_data = from_which_dict[key]
            if isinstance(param_data, dict):
                if 'unit' in param_data and param_data['unit'] == str(unit_dict[key]):
                    to_which_dict.update({
                        key : param_data['magnitude']* unit_dict[key]
                    })
                else:
                    raise AttributeError("ErrorSourceEvaluator : " + key + " should be with " + str(unit_dict[key]))
            else:
                to_which_dict.update({
                        key : param_data
                    })
                           
        return to_which_dict
        
                
    def get_specimen_parameters(self):
        # or in the run script call ErrorSourceEvaluator.measurement_param_dict
        return self.measurement_param_dict
    
    def get_pulse_parameters(self):
        return self.pulse_param_dict

    
########################################################################################################### ROI ####### 
class ErrorSourceEvaluatorROI(ErrorSourceEvaluator, DataGeneratorManualScanWithoutError):
    # constructor
    def __init__(self):
        super()__init__()
    
    def set_parameter_dataset(self):
        """ 
        What this function should do:
            1. choose appropriate specimen variables (except sub-vars)
            2. for loop with sub-vars
            3. for loop with main-vars
            4. call _set_file_names
            5. call all _register... fuctions 
            6. call the proper DataGenerator class
            7. call set_measurement_parameters() from the DataGenerator classes
        """
        pass
    
    def _register_error_source_variables(self, err_sources):
        """
        what this function should do:
            1. 
        """
        pass
    
######################################################################################################## Npoint #######
class ErrorSourceEvaluatorNpoint(ErrorSourceEvaluator, DataGeneratorManualScanWithoutError):
    # constructor
    def __init__(self):
        super()__init__()
        
    def set_parameter_dataset(self):
        """ 
        What this function should do:
            1. choose appropriate specimen variables (except sub-vars)
            2. for loop with sub-vars
            3. for loop with main-vars
            4. call _set_file_names
            5. call all _register... fuctions 
            6. call the proper DataGenerator class
            7. call set_measurement_parameters() from the DataGenerator classes
        """
        pass

    def _register_error_source_variables(self, err_sources):
        """
        what this function should do:
            1. 
        """
        pass
    
################################################################################################## Distribution #######
class ErrorSourceEvaluatorDistibution(ErrorSourceEvaluator, DataGeneratorManualScanWithoutError):
    # constructor
    def __init__(self):
        super()__init__()
    
    def set_parameter_dataset(self):
        """ 
        What this function should do:
            1. choose appropriate specimen variables (except sub-vars)
            2. for loop with sub-vars
            3. for loop with main-vars
            4. call _set_file_names
            5. call all _register... fuctions 
            6. call the proper DataGenerator class
            7. call set_measurement_parameters() from the DataGenerator classes
        """
        pass
 
    def _register_error_source_variables(self, err_sources):
        """
        what this function should do:
            1. 
        """
        pass
    

SyntaxError: invalid syntax (<ipython-input-50-509ffc0ca866>, line 244)